# Making CSV From Videos

In [ ]:
# All Imports
import cv2
import os
import mediapipe as mp
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Making CSV from landmark points
empty_list_big = []
empty_list_small = []
name_of_lists = []
for i in range(1,469):
    name_of_lists.append(str(i)+'x')
    name_of_lists.append(str(i)+'y')


fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.mp4', fourcc, 15.0, (640, 480))

#cap = cv2.VideoCapture("up.avi")
cap = cv2.VideoCapture(0)
pTime = 0

mpDraw = mp.solutions.drawing_utils
mpFaceMesh = mp.solutions.face_mesh
faceMesh = mpFaceMesh.FaceMesh(max_num_faces=1)
drawSpec = mpDraw.DrawingSpec(thickness=1, circle_radius=2)

while True:
    success, img = cap.read()
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = faceMesh.process(imgRGB)
    if results.multi_face_landmarks:
        for faceLms in results.multi_face_landmarks:
            mpDraw.draw_landmarks(img, faceLms, mpFaceMesh.FACE_CONNECTIONS,
                                  drawSpec,drawSpec)
            for id,lm in enumerate(faceLms.landmark):
                #print(lm)
                ih, iw, ic = img.shape
                x,y = int(lm.x*iw), int(lm.y*ih)
                #print(id+1,x,y)
                #empty_list_small.append(id+1)
                empty_list_small.append(x)
                empty_list_small.append(y)
            empty_list_big.append(empty_list_small)
            empty_list_small = []

    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
    cv2.putText(img, f'FPS: {int(fps)}', (20, 70), cv2.FONT_HERSHEY_PLAIN,
                3, (255, 0, 0), 3)
    cv2.imshow("Image", img)
    out.write(img)
    if cv2.waitKey(1) == 13:
        break

df = pd.DataFrame(empty_list_big, columns = name_of_lists)
df.to_csv("all_landmarks.csv", index= False)

cap.release()
cv2.destroyAllWindows()

# Capturing Image Frames For Dataset

In [ ]:
# Video capturing and dataset making 
# We will capture 2000 frames per classes
# The outputs classes are Up, Down, Left, Right, Upper Left, Upper Right, Lower Left, Lower Right, and Middle or Neutral 
# There should be a video that consits of all the classes for the experimental purposes 

In [ ]:
# Changeable Parameters
image_name = 'all_case'
max_count = 1000


#img = cv2.imread('1.jpg', 1)
#path = 'D:/OpenCV/Scripts/Images'
path = 'dataset/images/'
path = os.path.join(path, image_name)

if not os.path.isdir(path):
    os.mkdir(path)


#fourcc = cv2.VideoWriter_fourcc(*'XVID')
#out = cv2.VideoWriter(file_name, fourcc, 20.0, (640, 480))

#cap = cv2.VideoCapture("Videos/1.mp4")
cap = cv2.VideoCapture(0)

count = 0
while True:
    
    success, img = cap.read()
    
    count += 1
    
    cv2.imwrite(os.path.join(path, image_name + str(count)+'.jpg'), img)
    #out.write(img)
    #print(img.shape)
    if count%100 == 0:
        print(count)
    cv2.imshow("Image", img)
    if cv2.waitKey(1) == 13:
        break
    elif count == max_count:
        break

cap.release()
cv2.destroyAllWindows()

# Making CSV Files from the images

In [ ]:
def csv_maker(image_name):
    # Reading Folderwide all images
    #image_name = 'all_case'
    # Making CSV from landmark points
    empty_list_big = []
    empty_list_small = []
    name_of_lists = []
    
    for i in range(468):
        name_of_lists.append(str(i)+'x')
        name_of_lists.append(str(i)+'y')

    path = 'dataset/images/'
    path = os.path.join(path, image_name)
    images = os.listdir(path)

    mpDraw = mp.solutions.drawing_utils
    mpFaceMesh = mp.solutions.face_mesh
    faceMesh = mpFaceMesh.FaceMesh(max_num_faces=1)
    drawSpec = mpDraw.DrawingSpec(thickness=1, circle_radius=2)

    for every_image in images:
        img = cv2.imread(path+'/' + every_image)
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = faceMesh.process(imgRGB)
        if results.multi_face_landmarks:
            for faceLms in results.multi_face_landmarks:
                mpDraw.draw_landmarks(img, faceLms, mpFaceMesh.FACE_CONNECTIONS,
                                      drawSpec,drawSpec)
                for id,lm in enumerate(faceLms.landmark):
                    #print(lm)
                    ih, iw, ic = img.shape
                    x,y = int(lm.x*iw), int(lm.y*ih)
                    #print(id+1,x,y)
                    #empty_list_small.append(id+1)
                    empty_list_small.append(x)
                    empty_list_small.append(y)
                empty_list_big.append(empty_list_small)
                empty_list_small = []


    df = pd.DataFrame(empty_list_big, columns = name_of_lists)
    df["output"] = image_name
    df.to_csv("dataset/csv/"+ image_name+".csv", index = False)


    
    return str(image_name) + "done"
    



In [ ]:
class_list = ["down", "left", "lower_left", "lower_right","middle", "right", "up","upper_left", "upper_right"]
for every_class in class_list:
    print(csv_maker(every_class))
    

# Eye Bounding box applied to realtime video

In [ ]:
# Eye Bounding box applied to realtime video
# Parameters for eye landmark
# Landmark id starts from 0 ..please remember that 
r1 = 285 #336 # 285
r2 = 261 #340 # 261

l1 = 55 #70
l2 = 31 #196

empty_list_big = []
empty_list_small = []
name_of_lists = []

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.mp4', fourcc, 15.0, (640, 480))

#cap = cv2.VideoCapture("up.avi")
cap = cv2.VideoCapture(0)
pTime = 0

mpDraw = mp.solutions.drawing_utils
mpFaceMesh = mp.solutions.face_mesh
faceMesh = mpFaceMesh.FaceMesh(max_num_faces=1)
drawSpec = mpDraw.DrawingSpec(thickness=1, circle_radius=0)

while True:
    success, img = cap.read()
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = faceMesh.process(imgRGB)
    if results.multi_face_landmarks:
        for faceLms in results.multi_face_landmarks:
            #mpDraw.draw_landmarks(img, faceLms, mpFaceMesh.FACE_CONNECTIONS,
            #                      drawSpec,drawSpec)
            for id,lm in enumerate(faceLms.landmark):
                #print(lm)
                ih, iw, ic = img.shape
                x,y = int(lm.x*iw), int(lm.y*ih)
                #print(id+1,x,y)
                #empty_list_small.append(id+1)
                empty_list_small.append(x)
                empty_list_small.append(y)
            empty_list_big.append(empty_list_small)
            right_eye_x1 = empty_list_small[2 * r1] 
            right_eye_y1 = empty_list_small[2 * r1 + 1] 
            right_eye_x2 = empty_list_small[2 * r2 ] 
            right_eye_y2 = empty_list_small[2 * r2 + 1] 
            
            left_eye_x1_not = empty_list_small[2 * l1] 
            left_eye_y1_not = empty_list_small[2 * l1 + 1] 
            left_eye_x2_not = empty_list_small[2 * l2 ] 
            left_eye_y2_not = empty_list_small[2 * l2 + 1]  
            
            left_eye_x1 = left_eye_x2_not
            left_eye_y1 = left_eye_y1_not
            left_eye_x2 = left_eye_x1_not
            left_eye_y2 = left_eye_y2_not
            
            #print((right_eye_x1,right_eye_y1),(right_eye_x2,right_eye_y2))
            cv2.rectangle(img,(right_eye_x1,right_eye_y1),(right_eye_x2,right_eye_y2),(0,255,0),2)
            cv2.rectangle(img,(left_eye_x1,left_eye_y1),(left_eye_x2,left_eye_y2),(0,255,0),2)
            empty_list_small = []

    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
    cv2.putText(img, f'FPS: {int(fps)}', (20, 70), cv2.FONT_HERSHEY_PLAIN,
                3, (255, 0, 0), 3)
    cv2.imshow("Image", img)
    out.write(img)
    if cv2.waitKey(1) == 13:
        break

#df = pd.DataFrame(empty_list_big, columns = name_of_lists)
#df.to_csv("all_landmarks.csv", index= False)

cap.release()
cv2.destroyAllWindows()

# Image Cropping and Making a seperate dataset

In [ ]:
def eye_crop(img):
    # Parameters for eye landmark
    # Landmark id starts from 0 ..please remember that 
    r1 = 285 #336 # 285
    r2 = 261 #340 # 261

    l1 = 55 #70
    l2 = 31 #196

    empty_list_big = []
    empty_list_small = []
    name_of_lists = []

    mpDraw = mp.solutions.drawing_utils
    mpFaceMesh = mp.solutions.face_mesh
    faceMesh = mpFaceMesh.FaceMesh(max_num_faces=1)
    drawSpec = mpDraw.DrawingSpec(thickness=1, circle_radius=0)

    #success, img = cap.read()
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = faceMesh.process(imgRGB)
    if results.multi_face_landmarks:
        for faceLms in results.multi_face_landmarks:
            #mpDraw.draw_landmarks(img, faceLms, mpFaceMesh.FACE_CONNECTIONS,
            #                      drawSpec,drawSpec)
            for id,lm in enumerate(faceLms.landmark):
                #print(lm)
                ih, iw, ic = img.shape
                x,y = int(lm.x*iw), int(lm.y*ih)
                #print(id+1,x,y)
                #empty_list_small.append(id+1)
                empty_list_small.append(x)
                empty_list_small.append(y)
            empty_list_big.append(empty_list_small)
            right_eye_x1 = empty_list_small[2 * r1] 
            right_eye_y1 = empty_list_small[2 * r1 + 1] 
            right_eye_x2 = empty_list_small[2 * r2 ] 
            right_eye_y2 = empty_list_small[2 * r2 + 1] 
            
            left_eye_x1_not = empty_list_small[2 * l1] 
            left_eye_y1_not = empty_list_small[2 * l1 + 1] 
            left_eye_x2_not = empty_list_small[2 * l2 ] 
            left_eye_y2_not = empty_list_small[2 * l2 + 1]  
            
            left_eye_x1 = left_eye_x2_not
            left_eye_y1 = left_eye_y1_not
            left_eye_x2 = left_eye_x1_not
            left_eye_y2 = left_eye_y2_not
            
            img_left = img[left_eye_y1:left_eye_y2, left_eye_x1:left_eye_x2]
            img_right = img[right_eye_y1:right_eye_y2, right_eye_x1:right_eye_x2]
            
            #cv2.rectangle(img,(right_eye_x1,right_eye_y1),(right_eye_x2,right_eye_y2),(0,255,0),2)
            #cv2.rectangle(img,(left_eye_x1,left_eye_y1),(left_eye_x2,left_eye_y2),(0,255,0),2)
            empty_list_small = []
            
            return img_left, img_right



In [ ]:
def image_saver(image_name):
    
    
    
    base_dir = 'dataset/eye_cropped'
    training_path = os.path.join(base_dir, 'training')
    validation_path = os.path.join(base_dir, 'validation')
    
    if not os.path.isdir(os.path.join(training_path, image_name)):
        os.mkdir(os.path.join(training_path, image_name))
    if not os.path.isdir(os.path.join(validation_path, image_name)):
        os.mkdir(os.path.join(validation_path, image_name))
    
    input_image_path = 'dataset/images/'
    input_image_path = os.path.join(input_image_path, image_name)
    images = os.listdir(input_image_path)

    count = 1
    for every_image in images:
        img = cv2.imread(input_image_path + '/' + every_image)
        img_left, img_right = eye_crop(img)
        if count <= 800:
            cv2.imwrite(os.path.join(training_path, image_name + '/' + 'eye_left' + every_image ), img_left)
            cv2.imwrite(os.path.join(training_path, image_name + '/' +  'eye_right' + every_image), img_right)
        else:
            cv2.imwrite(os.path.join(validation_path, image_name + '/' +  'eye_left' + every_image ), img_left)
            cv2.imwrite(os.path.join(validation_path, image_name + '/' +  'eye_right' + every_image), img_right)
        if count % 100 == 0:
            print(image_name + 'saved' + str(count) )   
        count += 1

    return image_name + "done"
    



In [ ]:
class_list = ["up", "down", "left", "right", "upper_left", "upper_right", "lower_left", "lower_right", "middle"]
#for every_class in class_list:
print(image_saver(class_list[8]))
    

In [ ]:
exp_image = cv2.imread('dataset/images/experiment/IMG20210618175653.jpg')
img1, img2 = eye_crop(exp_image)
plt.imshow(img1)

In [ ]:
hello_world()

# Applying Mediapipe to single images and save outputs to inspect and decide which distances to use

In [ ]:
import cv2
import mediapipe as mp
import time
import os

import pandas as pd


mpDraw = mp.solutions.drawing_utils
mpFaceMesh = mp.solutions.face_mesh
faceMesh = mpFaceMesh.FaceMesh(max_num_faces=2)
drawSpec = mpDraw.DrawingSpec(thickness=1, circle_radius=0)



max_count = 10
mesh_image = 'facemesh'

path = 'dataset/images/experiment'
path2 = 'dataset/images/experiment_output'


if not os.path.isdir(path2):
    os.mkdir(path2)
    
images = os.listdir(path)

path_mesh = os.path.join(path2, mesh_image)

for every_image in images:
    img = cv2.imread(path+'/' + every_image)
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = faceMesh.process(imgRGB)
    if results.multi_face_landmarks:
        for faceLms in results.multi_face_landmarks:
            mpDraw.draw_landmarks(img, faceLms, None, 
                                  drawSpec, drawSpec)
            for id, lm in enumerate(faceLms.landmark):

                ih, iw, ic = img.shape
                x, y = int(lm.x*iw), int(lm.y*ih)
                cv2.putText(img, str(id), (x, y), cv2.FONT_HERSHEY_PLAIN,
                            0.5, (0, 255, 0), 1)
    cv2.imwrite(os.path.join(path2, mesh_image + str(every_image) +'.jpg'), img)